In [1]:
import Pkg
#Pkg.add("GLPK")
#Pkg.add("XLSX")
using Pkg
using JuMP
using GLPK
using Gurobi
using XLSX
using DataFrames

   Updating registry at `~/.julia/registries/General`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`


First, the data from excel files are called and saved in dataframe formats.

In [2]:
shared_cand = DataFrame(XLSX.readtable("CandLine.xlsx", "Taul1", header=true)...) #Dataframe of shared candidate lines
int_cand =  DataFrame(XLSX.readtable("CandLineInt.xlsx", "Taul1", header=true)...) #Dataframe of internal candidate lines
shared_ex =  DataFrame(XLSX.readtable("SharedEline.xlsx", "Taul1", header=true)...) #Dataframe of shared existing lines
int_ex = DataFrame(XLSX.readtable("Tran.xlsx", "Taul1", header=true)...) #Dataframe of internal existing lines
gen =  DataFrame(XLSX.readtable("Gen.xlsx" , "Taul1", header=true)...) #Dataframe of generators
load =  DataFrame(XLSX.readtable("Load.xlsx", "Taul1", header=true)...) #Dataframe of loads
##An example of the load dataframe can be found as follows

,zoneNum,lNodeID,P_load1,P_load2,P_load3,P_load4
,Any,Any,Any,Any,Any,Any
1,1,2,-21.7,-10.85,-16.275,-5.425
2,1,3,-94.2,-47.1,-70.65,-23.55
3,1,4,-47.8,-23.9,-35.85,-11.95
4,1,5,-7.6,-3.8,-5.7,-1.9
5,1,6,-11.2,-5.6,-8.4,-2.8
6,1,9,-29.5,-14.75,-22.125,-7.375
7,1,10,-9,-4.5,-6.75,-2.25
8,1,11,-3.5,-1.75,-2.625,-0.875
9,1,12,-6.1,-3.05,-4.575,-1.525


Then, some functions are defined which can separate the lines, generators, and zones of each zone

In [3]:
l(i) = load[load.zoneNum .== i, :] # load within zone i
g(i) = gen[gen.zoneNum .== i, :]   # generators within zone i
shared_c(i) = vcat(shared_cand[shared_cand.nodeZone1 .== i,:] , shared_cand[shared_cand.nodeZone2 .== i, :]) #shared candidate lines within zone i
int_c(i) = int_cand[int_cand.zoneNum .== i, :]   # number of internal candidate lines within zone i
shared_e(i) = vcat(shared_ex[shared_ex.nodeZone1 .== i,:] , shared_ex[shared_ex.nodeZone2 .== i, :]) #shared existing lines within zone i
int_e(i) =int_ex[int_ex.zoneNum .== i, :]       # internal existing lines within zone i
MC(i) = (g(i).C2 .* (g(i).PgMax .^ 2) .+ g(i).C1 .* g(i).PgMax .- g(i).C2 .*(g(i).PgMin .^ 2) .- g(i).C1 .* g(i).PgMin) ./ (g(i).PgMax .- g(i).PgMin) #Marginal cost of generators within zone i
bin_c(i) = (shared_cand.nodeZone1 .== i) + (shared_cand.nodeZone2 .== i) # A binary vector through which we can check if the shared candidate lines belong to zone i
bin_e(i) = (shared_ex.nodeZone1 .== i) + (shared_ex.nodeZone2 .== i) # A binary vector through which we can check if the shared existing lines belong to zone i

bin_e (generic function with 1 method)

As an example, the following shows the shared candidate lines that belong to zone 1:

In [4]:
shared_c(1)

,Sernum,tNodeID1,nodeZone1,tNodeID2,nodeZone2,reacT,ptMax,lifeTime,interestRate
,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,1,1,1,2,3,0.05917,100,50,0.05
2,3,10,1,4,3,0.19797,100,29,0.03
3,2,5,3,2,1,0.22304,100,57,0.045
4,4,19,2,14,1,0.0575,100,35,0.1


The folowing section shows the optimization problem that needs to be run by each transmission planner (TP). 
The optimization problem is written as a function of (i = the zone number, N = the number of nodes within zone i, flag = (if flag = 1, then we have binary decision variables, otherwise we have a relaxed problem), and cand_pi = the penalty vector associated with the candidate shared lines binary variables

In [5]:
function milp_hor_dist(i, N, flag, cand_pi, cand_mu, ex_mu)
    
    #Defining the model:
    
    Mod1 = Model(GLPK.Optimizer)
    
    #Defining variables
    
    @variable(Mod1,0 <= gen_var[1:N]) # generation at each node
    @variable(Mod1,0 <= shared_line_decision_var[1:nrow(shared_cand)] <= 1) #Decision variables for shared candidate lines
    @variable(Mod1,0 <= int_line_decision_var[1:nrow(int_c(i))] <= 1) #Decision variables for internal candidate lines
    @variable(Mod1,shared_cand_flow[1:nrow(shared_cand)])  #Power flowing into shared candidate lines 
    @variable(Mod1,int_cand_flow[1:nrow(int_c(i))]) #Power flowing into internal candidate lines 
    @variable(Mod1,0 <= shared_cand_angle[1:nrow(shared_cand),1:2]<= 2 * pi) #Phase angle decision for shared candidate lines
    @variable(Mod1,0 <= int_cand_angle[1:nrow(int_c(i)),1:2]<= 2 * pi) #Phase angle decision for internal candidate lines
    @variable(Mod1,shared_ex_flow[1:nrow(shared_ex)])  #Power flowing on shared existing shared lines
    @variable(Mod1,int_ex_flow[1:nrow(int_e(i))])  #Power flowing on internal existing lines
    @variable(Mod1,0 <= shared_ex_angle[1:nrow(shared_ex),1:2]<= 2 * pi) #Phase angle for existing shared lines
    @variable(Mod1,0 <= int_ex_angle[1:nrow(int_e(i)),1:2]<= 2 * pi) #Phase angle for internal existing lines
    #For those variables that have two columns:
    #The first column is fromnode
    #The second column is tonode
    
    
    #Defining the objective function
    @expression(Mod1, total_cost , sum((gen_var[n] .* sum((g(i).gNodeID .== n) .* MC(i))) for n in 1:N)
        .+ sum(shared_cand_flow[c] .* cand_mu[c] .+ cand_pi[c] .* shared_line_decision_var[c] .+  shared_line_decision_var[c] .* shared_cand.costPerCap[c] .* shared_cand.interestRate[c] 
                .*((1 + shared_cand.interestRate[c]) .^ shared_cand.lifeTime[c]) ./ (((1 + shared_cand.interestRate[c]) .^ shared_cand.lifeTime[c])-1) for c in 1:nrow(shared_cand))
        .+ sum(int_line_decision_var[c] .* int_c(i).costPerCap[c] .* int_c(i).interestRate[c] 
                .*((1 + int_c(i).interestRate[c]) .^ int_c(i).lifeTime[c]) ./ (((1 + int_c(i).interestRate[c]) .^ int_c(i).lifeTime[c])-1) for c in 1:nrow(int_c(i)))
        .+ sum(shared_ex_flow[h] .* ex_mu[h] for h in 1:nrow(shared_ex)))
    
    for n in 1:N
        # Power balance constraint for each node
        @constraint(Mod1, sum(g(i).gNodeID .== n) .* gen_var[n] .+ sum(l(i).P_load1 .* (l(i).lNodeID .== n)) .==
            sum((shared_cand.tNodeID1 .== n) .*bin_c(i) .* shared_cand_flow) .- sum((shared_cand.tNodeID2 .== n) .* bin_c(i) .* shared_cand_flow) .+
            sum((shared_ex.tNodeID1 .== n) .* bin_e(i) .* shared_ex_flow) .- sum((shared_ex.tNodeID2 .== n) .* bin_e(i) .* shared_ex_flow) .+
            sum((int_c(i).tNodeID1 .== n) .* int_cand_flow) .- sum((int_c(i).tNodeID2 .== n) .* int_cand_flow) .+
            sum((int_e(i).tNodeID1 .== n) .* int_ex_flow) .- sum((int_e(i).tNodeID2 .== n) .* int_ex_flow))
        #Lower limit for generation of each node
        @constraint(Mod1, sum(g(i).gNodeID .== n) .* gen_var[n] .<= sum((g(i).gNodeID .== n) .* g(i).PgMax))
        #Upper limit for generation of each node
        @constraint(Mod1, sum((g(i).gNodeID .== n) .* g(i).PgMin) .<= sum(g(i).gNodeID .== n) .* gen_var[n])
    end
    
    M=1000 #Big number
    
    #Constraints elated to the candidate shared lines:
    for c in 1:nrow(shared_cand) 
        if flag==1 
            #If flag = 1, our decision variables for constructing candidate shared lines would be binary
            @constraint(Mod1, shared_line_decision_var[c] in MOI.Integer())
        end
        #Associating the power flowing within the line with the phase angles of nodes:
        @constraint(Mod1,-M .* (1 .- shared_line_decision_var[c]) .<= shared_cand_flow[c] .- ((1 ./ shared_cand.reacT[c]) .* (shared_cand_angle[c,1] .- shared_cand_angle[c,2])))
        
        @constraint(Mod1, shared_cand_flow[c] .- ((1 ./ shared_cand.reacT[c]) .* (shared_cand_angle[c,1] .- shared_cand_angle[c,2])) .<= M .* (1 .- shared_line_decision_var[c]))
        #limiting the upper bound of power flow flowing within candidate shared lines
        @constraint(Mod1, shared_cand_flow[c] .<= shared_line_decision_var[c] .* shared_cand.ptMax[c])
        #Limiting the lower bound of power flowing within the candidate shared lines
        @constraint(Mod1, -shared_cand.ptMax[c] .* shared_line_decision_var[c] .<= shared_cand_flow[c])
    end
    
    #The same constraints related to the candidate internal lines:
    for c in 1:nrow(int_c(i))
        if flag==1
            @constraint(Mod1, int_line_decision_var[c] in MOI.Integer())
        end
        @constraint(Mod1,-M .* (1 .- int_line_decision_var[c]) .<= int_cand_flow[c] .- ((1 ./ int_c(i).reacT[c]) .* (int_cand_angle[c,1] .- int_cand_angle[c,2])))
        @constraint(Mod1, int_cand_flow[c] .- ((1 ./ int_c(i).reacT[c]) .* (int_cand_angle[c,1] .- int_cand_angle[c,2])) .<= M .* (1 .- int_line_decision_var[c]))
        @constraint(Mod1, int_cand_flow[c] .<= int_line_decision_var[c] .* int_c(i).ptMax[c])
        @constraint(Mod1, -int_c(i).ptMax[c] .*int_line_decision_var[c] .<= int_cand_flow[c])
    end
    
    #The same constraints related to the shared existing lines:
    for h in 1:nrow(shared_ex)
        @constraint(Mod1, shared_ex_flow[h] .== (1 ./ shared_ex.reacT[h]) .* (shared_ex_angle[h,1] .- shared_ex_angle[h,2]))
        @constraint(Mod1, shared_ex_flow[h] .<= shared_ex. ptMax[h])
        @constraint(Mod1, -shared_ex.ptMax[h] .<= shared_ex_flow[h])
    end
    #The same constraints related to the internal existing lines:
    
    for h in 1:nrow(int_e(i))
        @constraint(Mod1, int_ex_flow[h] .== (1 ./ int_e(i).reacT[h]) .* (int_ex_angle[h,1] .- int_ex_angle[h,2]))
        @constraint(Mod1, int_ex_flow[h] .<= int_e(i).ptMax[h])
        @constraint(Mod1, -int_e(i).ptMax[h] .<= int_ex_flow[h])
    end
    
    #The model objective is to minimize the total cost:
    
    @objective(Mod1, Min, total_cost)
    optimize!(Mod1)
    if termination_status(Mod1) == MOI.OPTIMAL #If the solution is optimal then it returns the variables
    
        shared_line_decision = value.(shared_line_decision_var)
        shared_cand_power = value.(shared_cand_flow)
        shared_cand_phase_angle = value.(shared_cand_angle)
        shared_ex_power = value.(shared_ex_flow)
        shared_ex_phase_angle = value.(shared_ex_angle)
        gen_power = value.(gen_var)
        int_line_decision = value.(int_line_decision_var)
        int_cand_power = value.(int_cand_flow)
        int_cand_phase_angle = value.(int_cand_angle)
        gen_power = value.(gen_var)
        obj_value = objective_value(Mod1)
        [obj_value, shared_line_decision, shared_cand_power,shared_cand_phase_angle,shared_ex_power,shared_ex_phase_angle,gen_power, int_line_decision, int_cand_power, int_cand_phase_angle, gen_power]

    end
end

milp_hor_dist (generic function with 1 method)

The above problem indicates the dual problem that should be solved by each TP. 

The second optimization problem should be solved by the marketoverseer or the transmission planner coordinator (TPC). The problem defines as a function of (fleg = 1 if we want to get binary decision variables for line construction, cand_zeta = the penalty vector associated with the candidate shared lines'phase angles, ex_zeta  = the penalty vector associated with the existing shared lines'phase angles, cand_pi = the penalty vector associated with the candidate shared lines binary variables  

In [6]:
function market_overseer(flag, cand_pi, cand_mu, ex_mu)
    
    #Model
    Mod2 = Model(GLPK.Optimizer)
    
    #Variables for shared candidate lines
    
    @variable(Mod2,0 <= shared_line_decision_var[1:nrow(shared_cand)] <= 1) #Decision variables
    @variable(Mod2,shared_cand_flow[1:nrow(shared_cand)]) #Flow within shared candidate lines
    @variable(Mod2,0 <= shared_cand_angle[1:nrow(shared_cand),1:2]<= 2*pi) #Phase angles of shared candidate lines

    #Variables for existing candidate lines
    @variable(Mod2,shared_ex_flow[1:nrow(shared_ex)]) #Flow within shared existing lines
    @variable(Mod2,0 <= shared_ex_angle[1:nrow(shared_ex),1:2]<= 2*pi) #Phase angles of shared existing lines
    
    #Objective function
    @expression(Mod2, mo_obj, sum(shared_cand_flow[c] .* cand_mu[c] .+ cand_pi[c] .* shared_line_decision_var[c] for c in 1:nrow(shared_cand))
        .+ .+ sum(shared_ex_flow[h] .* ex_mu[h] for h in 1:nrow(shared_ex)))
    
    M=1000
    
    #Constraints related to the shared candidate lines:
    for c in 1:nrow(shared_cand)
        if flag==1
            @constraint(Mod2, shared_line_decision_var[c] in MOI.Integer())
        end
        
        # Constraints that associate power flow with the phase angles of the nodes
        @constraint(Mod2,-M .* (1 .- shared_line_decision_var[c]) .<= shared_cand_flow[c] .- ((1 ./ shared_cand.reacT[c]) .* (shared_cand_angle[c,1] .- shared_cand_angle[c,2])))
        @constraint(Mod2, shared_cand_flow[c] .- ((1 ./ shared_cand.reacT[c]) .* (shared_cand_angle[c,1] .- shared_cand_angle[c,2])) .<= M .* (1 .- shared_line_decision_var[c]))
        @constraint(Mod2, shared_cand_flow[c] .<=   shared_cand.ptMax[c])
        #Limiting the lower bound of power flowing within the candidate shared lines
        @constraint(Mod2, -shared_cand.ptMax[c]  .<= shared_cand_flow[c])

    
    end 
    #The same constraints related to the shared candidate lines:
    for h in 1:nrow(shared_ex)
        @constraint(Mod2, shared_ex_flow[h] .== (1 ./ shared_ex.reacT[h]) .* (shared_ex_angle[h,1] .- shared_ex_angle[h,2]))
        @constraint(Mod2, shared_ex_flow[h] .<= shared_ex.ptMax[h])
        @constraint(Mod2, -shared_ex.ptMax[h] .<= shared_ex_flow[h])
    end
    
    @objective(Mod2, Max, mo_obj)
    optimize!(Mod2)
    
    if termination_status(Mod2) == MOI.OPTIMAL
    
        shared_line_decision = value.(shared_line_decision_var)
        shared_cand_power = value.(shared_cand_flow)
        shared_cand_phase_angle = value.(shared_cand_angle)
        shared_ex_power = value.(shared_ex_flow)
        shared_ex_phase_angle = value.(shared_ex_angle)
        obj_value = objective_value(Mod2)

        [obj_value, shared_line_decision, shared_cand_power,shared_cand_phase_angle,shared_ex_power,shared_ex_phase_angle]
    end
end
    
    

market_overseer (generic function with 1 method)

In the following section, we define the initial values for penalty parameters

In [7]:
cand_pi_1= zeros(30000,nrow(shared_cand))


cand_pi_2 = zeros(30000,nrow(shared_cand))


cand_pi_3 = zeros(30000,nrow(shared_cand))


cand_pi_mo = zeros(30000,nrow(shared_cand))

cand_mu_1= zeros(30000,nrow(shared_cand))


cand_mu_2 = zeros(30000,nrow(shared_cand))


cand_mu_3 = zeros(30000,nrow(shared_cand))


cand_mu_mo = zeros(30000,nrow(shared_cand))

ex_mu_1= zeros(30000,nrow(shared_ex))


ex_mu_2 = zeros(30000,nrow(shared_ex))


ex_mu_3 = zeros(30000,nrow(shared_ex))


ex_mu_mo = zeros(30000,nrow(shared_ex))


30000×8 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 ⋮                        ⋮         
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.

In [8]:
#Some initial values
f=1.1
u=100.01
u_new = 100.01
low=0.001
low_new=0.001
s=0.1

r1_bin=0.001 * ones(nrow(shared_cand))
r1= 0.001 * ones(nrow(shared_cand))
r2_bin= 0.001 * ones(nrow(shared_cand))
r2=0.001 * ones(nrow(shared_cand))
r3_bin=0.001 * ones(nrow(shared_cand))
r3=0.001 * ones(nrow(shared_cand))
r_mo_bin=0.001 * ones(nrow(shared_cand))
r_mo=0.001 * ones(nrow(shared_cand))
r_e_1 = 0.001 * ones(nrow(shared_ex))
r_e_2 = 0.001 * ones(nrow(shared_ex))
r_e_3 = 0.001 * ones(nrow(shared_ex))
int_line_1 = zeros(nrow(int_c(1)))
int_line_2 = zeros(nrow(int_c(2)))
int_line_3 = zeros(nrow(int_c(3)))

int_line_1_flow = zeros(nrow(int_c(1)))
int_line_2_flow = zeros(nrow(int_c(2)))
int_line_3_flow = zeros(nrow(int_c(3)))

gen_1 = zeros(nrow(g(1)))
gen_2 = zeros(nrow(g(2)))
gen_3 = zeros(nrow(g(3)))

r_e1 = zeros(nrow(shared_ex))
r_e2 = zeros(nrow(shared_ex))
r_e3 = zeros(nrow(shared_ex))

8-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [9]:
for j in 2:300     # It considers 1000 iteration
    if abs(f) > 0.0001  # f is our criterion showing the convergence of our problem
         #Solving the relaxed optimization problem (lower bound) for zone 1
        R1_1_l = milp_hor_dist(1, 14, 0, cand_pi_1[j,:], cand_mu_1[j,:], ex_mu_1[j,:])
        #Solving the relaxed optimization problem for zone 2
        R1_2_l = milp_hor_dist(2, 30, 0, cand_pi_2[j,:], cand_mu_2[j,:], ex_mu_2[j,:])
        #Solving the relaxed optimization problem for zone 3
        R1_3_l = milp_hor_dist(3, 5, 0, cand_pi_3[j,:], cand_mu_3[j,:], ex_mu_3[j,:])
        #Solving the optimization problem of the market overseer
        R2_l = market_overseer(0,cand_pi_mo[j,:], cand_mu_mo[j,:], ex_mu_mo[j,:])
        
        #Solving the optimization problem for zone 1
        R1_1_u = milp_hor_dist(1, 14, 1, cand_pi_1[j,:], cand_mu_1[j,:], ex_mu_1[j,:])
        #Solving the optimization problem for zone 2
        R1_2_u = milp_hor_dist(2, 30, 1, cand_pi_2[j,:], cand_mu_2[j,:], ex_mu_2[j,:])
        #Solving the optimization problem for zone 3
        R1_3_u = milp_hor_dist(3, 5, 1, cand_pi_3[j,:], cand_mu_3[j,:], ex_mu_3[j,:])
        #Solving the relaxed optimization problem for MO
        R2_u = market_overseer(1,cand_pi_mo[j,:], cand_mu_mo[j,:], ex_mu_mo[j,:])

        

        #Updating cand_pi for each zone and MO based on the results of shared candidate lines' decision variables
        cand_pi_1[j+1,:] .= cand_pi_1[j,:] .+ (1 * (R1_1_u[2] .- R2_u[2])) 
        cand_pi_2[j+1,:] .= cand_pi_2[j,:] .+ (1 * (R1_2_u[2] .- R2_u[2])) 
        cand_pi_3[j+1,:] .= cand_pi_3[j,:] .+ (1 * (R1_3_u[2] .- R2_u[2])) 
        cand_pi_mo[j+1,:] .= (cand_pi_1[j+1,:] .+ cand_pi_2[j+1,:] .+  cand_pi_3[j+1,:])
        
        cand_mu_1[j+1,:] .= cand_mu_1[j,:] .+ (0.01 * (R1_1_u[3] .- R2_u[3])) 
        cand_mu_2[j+1,:] .= cand_mu_2[j,:] .+ (0.01 * (R1_2_u[3] .- R2_u[3])) 
        cand_mu_3[j+1,:] .= cand_mu_3[j,:] .+ (0.01 * (R1_3_u[3] .- R2_u[3])) 
        cand_mu_mo[j+1,:] .=  (cand_mu_1[j+1,:] .+ cand_mu_2[j+1,:] .+  cand_mu_3[j+1,:])
        
        ex_mu_1[j+1,:] .= ex_mu_1[j,:] .+ (0.01 * (R1_1_u[5] .- R2_u[5])) 
        ex_mu_2[j+1,:] .= ex_mu_2[j,:] .+ (0.01 * (R1_2_u[5] .- R2_u[5])) 
        ex_mu_3[j+1,:] .= ex_mu_3[j,:] .+ (0.01 * (R1_3_u[5] .- R2_u[5])) 
        ex_mu_mo[j+1,:] .= (ex_mu_1[j+1,:] .+ ex_mu_2[j+1,:] .+  ex_mu_3[j+1,:])
        
        
        #u = the sum of objective values after solving the zonal optimization problem
        u_new = R1_1_u[1] + R1_2_u[1] + R1_3_u[1] + R2_u[1]
        
        #low = the sum of objective values after solving the relaxed zonal optimization problem
        low_new = R1_1_l[1] + R1_2_l[1] + R1_3_l[1] + R2_l[1]
       
         #Stopping criterion
        f = 1 - (low_new / u_new)
        
        
        r1_bin = R1_1_u[2]
        r1 = R1_1_u[3]
        r2_bin = R1_2_u[2]
        r2 = R1_2_u[3]
        r3_bin = R1_3_u[2]
        r3 = R1_3_u[3]
        r_mo_bin = R2_u[2]
        r_mo = R2_u[3]
        r_e_1 = R1_1_u[6]
        r_e_2 = R1_2_u[6]
        r_e_3 = R1_3_u[6]
        r_e1 = R1_1_u[5]
        r_e2 = R1_2_u[5]
        r_e3 = R1_3_u[5]
        int_line_1 = R1_1_u[8]
        int_line_2 = R1_2_u[8]
        int_line_3 = R1_3_u[8]
        int_line_1_flow = R1_1_u[9]
        int_line_2_flow = R1_2_u[9]
        int_line_3_flow = R1_3_u[9]
        gen_1 = R1_1_u[11]
        gen_2 = R1_2_u[11]
        gen_3 = R1_3_u[11]
        print('*') 
        
    else
        #If our criterion is met, then print these values:
        print(low_new)
        print(u_new)
        print(int_line_1)
        print(int_line_2)
        print(int_line_3)
        print(j)
        print(f) #criterion
        print(r1_bin) #binary decisions of shared candidate lines (zone 1)
        print(r1) # power flow of shared candidate lines (zone 1)
        print(r2_bin) #binary decisions of shared candidate lines (zone 2)
        print(r2) # power flow of shared candidate lines (zone 2)
        print(r3_bin) #binary decisions of shared candidate lines (zone 3)
        print(r3) # power flow of shared candidate lines (zone 3)
        print(r_mo_bin)  #binary decisions of shared candidate lines (MO)
        print(r_mo) # power flow of shared candidate lines (MO)
        print(r_e_1) # angle of existing candidate lines (zone 1)
        print(r_e_2)  # angle of existing candidate lines (zone 2)
        print(r_e_3) # angle of existing candidate lines (zone 3)
        break
    end
end



***********************************************************************************************************************************************************************************************************************************************************************************************************

In [10]:
f #The criteria

0.016236891133265652

In [11]:
r1_bin

6-element Vector{Float64}:
 1.0
 0.0
 0.0
 1.0
 1.0
 0.0

In [12]:
r2_bin

6-element Vector{Float64}:
 1.0
 0.0
 0.0
 1.0
 1.0
 0.0

In [13]:
r3_bin #Binary decisions for constructing shared lines solved by TP 3

6-element Vector{Float64}:
 1.0
 0.0
 0.0
 1.0
 1.0
 0.0

In [14]:
r_mo_bin #Binary decisions for constructing shared lines solved by TPC or MO

6-element Vector{Float64}:
 1.0
 0.0
 0.0
 1.0
 1.0
 0.0

In [15]:
r1 #The value obtained by TP 1 and indicating the power flowing into the shared lines

6-element Vector{Float64}:
  99.99999999999997
  -0.0
  -0.0
 100.0
  38.03380936549385
  -0.0

In [16]:
r2 #The value obtained by TP 2 and indicating the power flowing into the shared lines

6-element Vector{Float64}:
 99.99999999999997
 -0.0
  0.0
 99.99999999999997
 38.03380936549388
 -0.0

In [17]:
r3 #The value obtained by TP 1 and indicating the power flowing into the shared lines

6-element Vector{Float64}:
 99.99999999999997
 -0.0
  0.0
 99.99999999999997
 38.03380936549388
  0.0

In [18]:
r_e_1

8×2 Matrix{Float64}:
 6.28319  0.0
 6.28319  0.0
 0.0      6.28319
 6.28319  0.0
 5.99     0.0
 6.28319  0.0
 6.28319  0.0
 0.0      6.28319

In [19]:
r_e_2

8×2 Matrix{Float64}:
 6.28319  0.0
 6.28319  0.0
 0.0      6.28319
 6.28319  0.0
 5.99     0.0
 6.28319  0.0
 6.28319  0.0
 0.0      6.28319

In [20]:
r_e_3

8×2 Matrix{Float64}:
 6.28319  0.0
 6.28319  0.0
 0.0      6.28319
 0.0      6.28319
 3.91402  0.0
 6.28319  0.0
 6.28319  0.0
 0.0      6.28319

In [21]:
u_new

6383.323251797913

In [22]:
low_new

6279.677927090027

In [23]:
int_line_1

10-element Vector{Float64}:
 0.0
 1.0
 1.0
 0.0
 1.0
 1.0
 0.0
 1.0
 0.0
 1.0

In [24]:
int_line_2

18-element Vector{Float64}:
 1.0
 0.0
 0.0
 0.0
 0.0
 1.0
 1.0
 0.0
 0.0
 1.0
 1.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [25]:
int_line_3

2-element Vector{Float64}:
 0.0
 0.0

In [26]:
gen_1

14-element Vector{Float64}:
 135.7289433067423
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0

In [27]:
show(stdout, "text/plain", gen_2)

30-element Vector{Float64}:
 32.18584707130596
 80.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
 40.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
 37.01415292869409
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0

In [28]:
gen_3

5-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0

In [29]:
int_line_1_flow

10-element Vector{Float64}:
   0.0
  33.670317828676275
 -36.13518120071069
   0.0
  35.045145548889934
 -30.045836396229788
   0.0
  24.93129635417654
   0.0
  13.761720136141406

In [30]:
int_line_2_flow

18-element Vector{Float64}:
 -83.88765430146304
   0.0
  -0.0
   0.0
  -0.0
 -31.62146606532253
 -31.463121217724506
  -0.0
   0.0
 -19.67240086752782
 -47.730506204577274
  -7.105427357601002e-15
   0.0
   0.0
   0.0
   0.0
   0.0
  -0.0

In [31]:
int_line_3_flow

2-element Vector{Float64}:
 0.0
 0.0

In [32]:
r_e1

8-element Vector{Float64}:
  23.27105669325772
  19.086225112939204
 -30.207621669132635
  11.300692998524433
 100.0
  31.41592653589793
  57.11986642890534
 -30.207621669132635

In [33]:
r_e2

8-element Vector{Float64}:
  23.27105669325772
  19.086225112939204
 -30.207621669132635
  11.300692998524433
 100.0
  31.41592653589793
  57.11986642890534
 -30.207621669132624

In [34]:
r_e3

8-element Vector{Float64}:
  23.27105669325772
  19.086225112939204
 -30.207621669132635
 -11.300692998524434
  65.34264242947552
  31.41592653589793
  57.11986642890534
 -30.207621669132624